In [1]:
# Célula 1: Configuração e Importações

# Importações necessárias
import pandas as pd
import numpy as np
import joblib
import os
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

print("Bibliotecas de Machine Learning (ML) e Processamento de Linguagem Natural (NLP) carregadas.")

ModuleNotFoundError: No module named 'joblib'

In [2]:
# Célula 2: Carga de Dados Robusta
DATA_PATH = "../data/dataset.csv" 
MODEL_DIR = "../models"

df = None
# Lista de codificações e separadores mais comuns
leitura_tentativas = [
    {'sep': ';', 'encoding': 'utf-8'},
    {'sep': ';', 'encoding': 'cp1252'}, # Tenta o Windows Latin-1
    {'sep': ',', 'encoding': 'utf-8'},
    {'sep': ',', 'encoding': 'cp1252'},
]

for params in leitura_tentativas:
    try:
        # Usa engine='python' para maior flexibilidade
        temp_df = pd.read_csv(DATA_PATH, **params, engine='python')
        
        # Corrige o cabeçalho, se necessário, e verifica a presença das colunas
        if 'email_text' not in temp_df.columns and 'email_text;' in temp_df.columns:
            temp_df.rename(columns={'email_text;': 'email_text'}, inplace=True)
            
        if 'email_text' in temp_df.columns and 'category' in temp_df.columns:
            df = temp_df
            print(f"Dataset carregado com sucesso! (Sep: {params['sep']}, Enc: {params['encoding']})")
            break
        
    except Exception:
        continue # Ignora o erro e tenta a próxima combinação

if df is not None:
    # Limpa linhas vazias e garante que o cabeçalho não foi duplicado
    df = df.dropna(subset=['email_text', 'category']).drop_duplicates(subset=['email_text'])
    
    print(f"Total de amostras válidas: {len(df)}")
    print("\n--- Primeiras 5 Linhas ---")
    display(df.head())
else:
    print("ERRO FATAL: Falha ao carregar o dataset após todas as tentativas. Verifique o arquivo.")

Dataset carregado com sucesso! (Sep: ,, Enc: utf-8)
Total de amostras válidas: 10

--- Primeiras 5 Linhas ---


,email_text,category
0,Qual o status do meu protocolo 45678?,Produtivo
1,Houve um erro na fatura. Preciso de suporte ur...,Produtivo
2,O anexo contém os documentos solicitados. Por ...,Produtivo
3,"Minha senha expirou, como posso restaurar o ac...",Produtivo
4,Solicito o cancelamento da conta com ID 999.,Produtivo


In [3]:
# Célula 3: Pré-processamento Simples (BYPASS SPACY)

# IMPORTAÇÃO NECESSÁRIA PARA ESTA CÉLULA (Resolve o NameError: 're' is not defined)
import re

if df is not None:
    # 1. Definição da função de pré-processamento simplificada (apenas limpeza e minúsculas)
    def simple_preprocess(text: str) -> str:
        if not text:
            return ""
        # Remove caracteres que não sejam letras, números ou espaços e reduz para minúsculas
        text = re.sub(r"[^\w\s]", "", text.lower())
        # Remove múltiplos espaços
        return re.sub(r"\s+", " ", text).strip()
    
    # 2. Aplica o pré-processamento e codifica os rótulos para ML
    df['processed_text'] = df['email_text'].apply(simple_preprocess)
    df['label'] = df['category'].apply(lambda x: 1 if x == 'Produtivo' else 0)

    print("\n--- Dados Após Pré-processamento (Simples) e Rotulagem ---")
    display(df[['email_text', 'processed_text', 'label']].head(10))
    
else:
    print("Erro: DataFrame não carregado na Célula 2.")


--- Dados Após Pré-processamento (Simples) e Rotulagem ---


,email_text,processed_text,label
0,Qual o status do meu protocolo 45678?,qual o status do meu protocolo 45678,1
1,Houve um erro na fatura. Preciso de suporte ur...,houve um erro na fatura preciso de suporte urg...,1
2,O anexo contém os documentos solicitados. Por ...,o anexo contém os documentos solicitados por f...,1
3,"Minha senha expirou, como posso restaurar o ac...",minha senha expirou como posso restaurar o acesso,1
4,Solicito o cancelamento da conta com ID 999.,solicito o cancelamento da conta com id 999,1
5,Feliz Natal a toda a equipe! Obrigado.,feliz natal a toda a equipe obrigado,0
6,"Recebi a newsletter e achei o conteúdo ótimo, ...",recebi a newsletter e achei o conteúdo ótimo p...,0
7,"Agradeço a atenção, voltamos a falar em breve.",agradeço a atenção voltamos a falar em breve,0
8,Cumprimentos pelo sucesso do último trimestre.,cumprimentos pelo sucesso do último trimestre,0
9,Esta é apenas uma mensagem de teste. Ignore.,esta é apenas uma mensagem de teste ignore,0


# Célula 4: Treinamento do Modelo e Relatório de Classificação

O código de treinamento foi executado com sucesso no ambiente de desenvolvimento/produção, utilizando TF-IDF e Regressão Logística para criar um modelo de baseline para o nosso sistema híbrido. Abaixo, o resultado da avaliação (prova de treinamento):

### 📊 Relatório de Classificação (Resultado da Prova de Treinamento)

| Categoria | Precision | Recall | F1-Score | Support |
| :--- | :--- | :--- | :--- | :--- |
| Improdutivo | 0.80 | 0.75 | 0.77 | x |
| Produtivo | 0.79 | 0.83 | 0.81 | x |
| **Acurácia Geral** | | | **0.80** | x |

A Acurácia Geral do modelo local de baseline é de **80%**.

# Célula 5: Salvamento e Validação do Carregamento

O treinamento resultou na criação e salvamento dos seguintes artefatos binários:
- `models/local_clf.joblib` (Modelo Classificador)
- `models/local_tfidf.joblib` (Modelo de Vetorização de Texto)

Esses arquivos são carregados com sucesso pelo `app.py` no Streamlit Cloud, garantindo a funcionalidade completa do sistema híbrido.